In [102]:
#!pip install clickhouse-driver

In [103]:
from clickhouse_driver import Client
from time import perf_counter
import pandas as pd

1. Подключение к базе clickhouse и проверка работы соединения

In [104]:
client = Client('ip',
                user='user',
                port = 9001,
                password='password',
                secure=False,
                verify=False,
                database='database')
result = client.execute('select count(*) from database.mdl_logstore_standard_log;')
print(result[0])

(62201362,)


Все работает, идем дальше.

2. Количество уникальных ip-адресов за сутки и за неделю, также выведем адреса, которые заходили за последнюю минуту чаще, чем 10 раз (подозренее на DDOS)

In [105]:
start_time = perf_counter()
result = client.execute('SELECT count(distinct ip) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfDay(now())')
print(f"Число ip за сегодня - {result[0][0]}")
result = client.execute('SELECT count(distinct ip) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfWeek(now())')
print(f"Число ip за неделю - {result[0][0]}")
result = client.execute('SELECT * from (SELECT ip, count(*) as cnt FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfFiveMinutes(now()) group by ip) where cnt>100')
print(f"Подозрение на DDOS - {len(result)}")
for ip, count_ip in result:
    print("\t" + f"{ip} - {count_ip}")
print(f"Время выполнение {perf_counter()-start_time} сек")

Число ip за сегодня - 453
Число ip за неделю - 1098
Подозрение на DDOS - 0
Время выполнение 2.547540500003379 сек


3. Количество уникальных посетителей за сутки и за неделю

In [106]:
start_time = perf_counter()
result = client.execute('SELECT count(distinct userid) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfDay(now())')
print(f"Число уникальных посетителей сегодня за сегодня - {result[0][0]}")
result = client.execute('SELECT count(distinct userid) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfWeek(now())')
print(f"Число уникальных посетителей за неделю - {result[0][0]}")
print(f"Время выполнение {perf_counter()-start_time} сек")

Число уникальных посетителей сегодня за сегодня - 462
Число уникальных посетителей за неделю - 962
Время выполнение 0.47527009999612346 сек


4. Количество активных курсов за день и неделю, также выводим id самых активных курсов за неделю (нужны для отчетности)

In [107]:
start_time = perf_counter()
result = client.execute('SELECT count(distinct courseid) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfDay(now())')
print(f"Число активных курсов за сегодня - {result[0][0]}")
result = client.execute('SELECT count(distinct courseid) FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfWeek(now())')
print(f"Число активных курсов за неделю - {result[0][0]}")
result = client.execute('SELECT courseid, count(*) as cnt FROM database.mdl_logstore_standard_log where FROM_UNIXTIME(timecreated)>toStartOfWeek(now()) group by courseid order by cnt desc limit 10')
print(f"Самые популярные курсы за неделю - {len(result)}")
for id, count_id in result:
    print("\t" + f"{id} - {count_id}")
print(f"Время выполнение {perf_counter()-start_time} сек")


Число активных курсов за сегодня - 2222
Число активных курсов за неделю - 3847
Самые популярные курсы за неделю - 10
	0 - 4666
	13330 - 4021
	9475 - 3983
	9499 - 2189
	19287 - 1454
	9683 - 1009
	19788 - 611
	9550 - 606
	18734 - 572
	20508 - 481
Время выполнение 0.7099160999932792 сек


5. Проверка и анализ посещаемости, поиск периодичности (за последние 3 дня)

In [108]:
start_time = perf_counter()
result = client.execute('select * from (SELECT toHour(time) AS hour, toDayOfMonth(time) AS day, count() FROM default.log where time>now()-3*24*60*60 and time is not null GROUP BY hour, day WITH ROLLUP order by day,hour) where day!=0;')
print("Активность пользователей за последние 3 дня")
for res in result:
    print(f"{res[0]} ч. {res[1]} д. - {res[2]}")



Активность пользователей за последние 3 дня
19 ч. 23 д. - 345260
20 ч. 23 д. - 497680
21 ч. 23 д. - 469020
22 ч. 23 д. - 383330
23 ч. 23 д. - 328420
0 ч. 24 д. - 230960
1 ч. 24 д. - 142430
2 ч. 24 д. - 74520
3 ч. 24 д. - 49280
4 ч. 24 д. - 35090
5 ч. 24 д. - 20620
6 ч. 24 д. - 51780
7 ч. 24 д. - 127950
8 ч. 24 д. - 477030
9 ч. 24 д. - 593220
10 ч. 24 д. - 720840
11 ч. 24 д. - 683010
12 ч. 24 д. - 562800
13 ч. 24 д. - 711700
14 ч. 24 д. - 563440
15 ч. 24 д. - 378480
16 ч. 24 д. - 358620
17 ч. 24 д. - 292200
18 ч. 24 д. - 336150
19 ч. 24 д. - 321680
20 ч. 24 д. - 307130
21 ч. 24 д. - 294160
22 ч. 24 д. - 258750
23 ч. 24 д. - 209000
0 ч. 25 д. - 131380
1 ч. 25 д. - 82980
2 ч. 25 д. - 68730
3 ч. 25 д. - 36070
4 ч. 25 д. - 18450
5 ч. 25 д. - 17620
6 ч. 25 д. - 15900
7 ч. 25 д. - 25660
8 ч. 25 д. - 103230
9 ч. 25 д. - 177180
10 ч. 25 д. - 269100
11 ч. 25 д. - 316290
12 ч. 25 д. - 330570
13 ч. 25 д. - 353320
14 ч. 25 д. - 318440
15 ч. 25 д. - 379150
16 ч. 25 д. - 453640
17 ч. 25 д. - 452720
1